In [1]:
!pip install pyspark

     |████████████████████████████████| 204.2 MB 28 kB/s s eta 0:00:01
     |████████████████████████████████| 198 kB 63.1 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612244 sha256=ca2eb81ac08d9c76b9138098f6091b03fff1a4c02d332c8d92c83248746f507a
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
from pyspark.mllib.stat import Statistics
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.types import StringType, DoubleType, IntegerType
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
df_train = spark.read.csv("../input/home-credit-default-risk/application_train.csv",inferSchema="true", header="true")

NameError: name 'spark' is not defined

# Create Correlation heatmap

In [ ]:
#Sort out numerical columns
double_cols = [f.name for f in df_train.schema.fields if isinstance(f.dataType, DoubleType)]
int_cols = [f.name for f in df_train.schema.fields if isinstance(f.dataType, IntegerType)]
num_cols = int_cols+double_cols

In [ ]:
# convert to vector column first
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=num_cols, outputCol=vector_col)
df_vector = assembler.setHandleInvalid("skip").transform(df_train).select(vector_col)

# get correlation matrix
matrix = Correlation.corr(df_vector, vector_col)

In [ ]:
matrix = matrix.collect()[0][0]
corrmatrix = matrix.toArray().tolist()

In [ ]:
plt.figure(figsize=(50,50))
sns.heatmap(corrmatrix)

# Apply OneHotEncoder

In [ ]:
#Sort out categorical columns
str_cols = [f.name for f in df_train.schema.fields if isinstance(f.dataType, StringType)]

In [ ]:
stages = []
for categoricalCol in str_cols:
    #create a string indexer for those categorical values and assign a new name including the word 'Index'
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')

    #append the string Indexer to our list of stages
    stages += [stringIndexer]

#Create the pipeline. Assign the satges list to the pipeline key word stages
pipeline = Pipeline(stages = stages)
#fit the pipeline to our dataframe
pipelineModel = pipeline.fit(df_train)
#transform the dataframe
df= pipelineModel.transform(df_train)

In [ ]:
#create input columns and output columns
str_cols_index = []
str_cols_vec = []
for categoricalCol in str_cols:
    str_cols_index.append(categoricalCol + 'Index')
    str_cols_vec.append(categoricalCol + 'vec')

In [ ]:
encoder = OneHotEncoder(inputCols=str_cols_index,
                        outputCols=str_cols_vec)
model = encoder.fit(df)
encoded = model.transform(df)
encoded.columns
#Not yet complete